# RunnableBranch

`RunnableBranch` 는 입력에 따라 동적으로 로직을 라우팅할 수 있는 강력한 도구입니다. <br>이를 통해 개발자는 **입력 데이터의 특성에 기반하여 다양한 처리 경로를 유연하게 정의** 할 수 있습니다.

`RunnableBranch` 는 복잡한 의사 결정 트리를 간단하고 직관적인 방식으로 구현할 수 있도록 도와줍니다. <br>이는 코드의 가독성과 유지보수성을 크게 향상시키며, 로직의 모듈화와 재사용성을 촉진합니다.

또한, `RunnableBranch` 는 런타임에 동적으로 분기 조건을 평가하고 적절한 처리 루틴을 선택할 수 있어, <br>시스템의 적응력과 확장성을 높여줍니다.

이러한 특징들로 인해 RunnableBranch는 다양한 도메인에서 활용될 수 있으며, <br>특히 입력 데이터의 다양성과 변동성이 큰 애플리케이션 개발에 매우 유용합니다. <br>`RunnableBranch` 를 효과적으로 활용하면 코드의 복잡성을 줄이고, 시스템의 유연성과 성능을 향상시킬 수 있습니다.


## 입력에 따른 동적 로직 라우팅

LangChain Expression Language에서 라우팅을 수행하는 방법에 대해 다룹니다.

라우팅을 통해 이전 단계의 출력이 다음 단계를 정의하는 비결정적 체인을 생성할 수 있습니다. 라우팅은 LLM과의 상호 작용에 구조와 일관성을 제공하는 데 도움이 됩니다.

라우팅을 수행하는 방법에는 두 가지가 있습니다.

1. `RunnableLambda` 에서 조건부로 실행 가능한 객체를 반환 (권장)
2. `RunnableBranch` 사용

두 가지 방법 모두 첫 번째 단계에서 입력 질문을 `수학`, `과학` 또는 `기타`에 대한 것으로 분류한 다음, 해당 프롬프트 체인으로 라우팅하는 두 단계 시퀀스를 사용하여 설명하겠습니다.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH13-LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH13-LCEL-Advanced


## 간단한 예시

먼저, 들어오는 질문이 `수학`, `과학`, 또는 `기타` 중 하나로 분류하는 Chain을 생성하겠습니다.


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

# 체인을 생성합니다.
chain = (
    prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()  # 문자열 출력 파서를 사용합니다.
)

생성한 chain을 사용하여 질문을 분류합니다.


In [5]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "2+2 는 무엇인가요?"})

'수학'

In [6]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})

'과학'

In [7]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "Google은 어떤 회사인가요?"})

'기타'

이제 세 개의 하위 체인을 생성해 보겠습니다.


In [8]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

## 사용자 정의 함수 사용하기

LangChain 공식 도큐먼트에서 권장하는 방식이며, 서로 다른 출력 간의 라우팅을 위해 **사용자 정의 함수**를 `RunnableLambda` 로 래핑하여 활용할 수도 있습니다.




In [9]:
def route(info):
    # 주제에 "수학"이 포함되어 있는 경우
    if "수학" in info["topic"].lower():
        # math_chain 반환
        return math_chain
    # 주제에 "과학"이 포함되어 있는 경우
    elif "과학" in info["topic"].lower():
        # art_chain을 반환
        return science_chain
    # 그 외의 경우
    else:
        # general_chain을 반환
        return general_chain

In [10]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic": chain, "question": itemgetter("question")}
    | RunnableLambda(
        # 경로를 지정하는 함수를 인자로 전달합니다.
        route
    )
    | StrOutputParser()
)

In [11]:
# 수학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 함수의 변화율과 누적량을 다룹니다. 미분은 함수의 순간적인 변화율을 측정하며, 적분은 이러한 변화율을 통해 전체를 누적하여 계산하는 방법입니다. 미적분은 물리학, 공학, 경제학 등 다양한 분야에서 중요한 역할을 하며, 함수의 기울기, 넓이, 부피 등을 구하는 데 사용됩니다. 이 두 가지 개념은 서로 긴밀하게 연결되어 있으며, 미적분학의 기본 정리에 의해 미분과 적분이 서로 반대의 과정임을 나타냅니다.'

In [12]:
# 과학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "중력은 어떻게 작용하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력은 물체 간의 상호작용으로 발생하는 힘이며, 모든 물체는 서로를 끌어당기는 성질을 가지고 있습니다. 뉴턴의 만유인력 법칙에 따르면, 두 물체 사이의 중력은 그들의 질량의 곱에 비례하고, 두 물체 간의 거리의 제곱에 반비례합니다. 즉, 질량이 클수록 중력이 강해지고, 거리가 멀어질수록 중력은 약해집니다. 이로 인해 지구가 사물을 끌어당기고, 행성들이 태양 주위를 돌며, 우주에서 여러 천체가 서로 영향을 주고받는 현상을 설명할 수 있습니다.'

In [13]:
# 기타 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG( Retrieval Augmented Generation)은 정보를 검색하고 그 결과를 바탕으로 텍스트를 생성하는 모델입니다. 이 방법은 사전 훈련된 언어 모델과 외부 데이터베이스를 결합하여, 보다 정확하고 관련성 높은 응답을 생성할 수 있도록 합니다. RAG는 특히 질문 응답 시스템, 대화형 AI 등에서 활용됩니다.'

## RunnableBranch

`RunnableBranch`는 입력값에 따라 실행할 조건과 Runnable을 정의할 수 있는 특별한 유형의 `Runnable` 입니다.

다만, 위에서 설명한 사용자 정의 함수로 구현할 수 없는 기능을 제공하지는 않으므로, 사용자 정의 함수를 사용하는 것이 좋습니다.

**문법**

- `RunnableBranch`는 (조건, Runnable) 쌍의 리스트와 기본 Runnable로 초기화됩니다.
- 호출 시 전달된 입력값을 각 조건에 전달하여 분기를 선택합니다.
- True로 평가되는 첫 번째 조건을 선택하고, 해당 조건에 해당하는 Runnable을 입력값과 함께 실행합니다.
- 제공된 조건과 일치하는 것이 없으면 **기본 `Runnable`** 을 실행합니다.

In [14]:
from operator import itemgetter
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    # 주제에 "수학"이 포함되어 있는지 확인하고, 포함되어 있다면 math_chain을 실행합니다.
    (lambda x: "수학" in x["topic"].lower(), math_chain),
    # 주제에 "과학"이 포함되어 있는지 확인하고, 포함되어 있다면 science_chain을 실행합니다.
    (lambda x: "과학" in x["topic"].lower(), science_chain),
    # 위의 조건에 해당하지 않는 경우 general_chain을 실행합니다.
    general_chain,
)
# 주제와 질문을 입력받아 branch를 실행하는 전체 체인을 정의합니다.
full_chain = (
    {"topic": chain, "question": itemgetter("question")} | branch | StrOutputParser()
)

In [15]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 변화와 기울기를 다루는 중요한 개념입니다. 미적분은 두 가지 주요 부분으로 나눌 수 있습니다: 미분과 적분입니다.\n\n미분은 함수의 변화율을 구하는 과정으로, 어떤 함수의 기울기를 측정하여 특정 지점에서의 순간적인 변화율을 나타냅니다. 예를 들어, 자동차의 속도는 위치의 시간에 대한 미분을 통해 구할 수 있습니다.\n\n반면, 적분은 함수의 구간에 걸쳐 면적을 구하는 과정으로, 주어진 함수의 전체적인 합을 계산합니다. 예를 들어, 물체가 일정 시간 동안 이동한 거리나, 곡선 아래의 면적을 구하는 데 사용됩니다.\n\n미적분은 자연현상, 물리학, 공학 등 다양한 분야에서 발생하는 문제를 해결하는 데 필수적인 도구로 자리잡고 있습니다.'

In [16]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "중력 가속도는 어떻게 계산하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력 가속도는 물체가 지구의 중력에 의해 얼마나 빠르게 가속되는지를 나타내는 값으로, 일반적으로 약 9.81 m/s²로 알려져 있습니다. 이를 계산하기 위해서는 뉴턴의 만유인력 법칙을 사용할 수 있습니다. \n\n만유인력 법칙에 따르면, 두 질량 \\( m_1 \\)과 \\( m_2 \\) 사이의 중력 \\( F \\)는 다음과 같이 표현됩니다:\n\n\\[ F = G \\frac{m_1 m_2}{r^2} \\]\n\n여기서 \\( G \\)는 만유인력 상수이며, \\( r \\)은 두 질량 사이의 거리입니다. 지구의 경우, 물체의 질량 \\( m \\)와 지구의 질량 \\( M \\)을 고려하면, 중력 가속도 \\( g \\)는 다음과 같이 표현할 수 있습니다:\n\n\\[ g = \\frac{F}{m} = G \\frac{M}{r^2} \\]\n\n이렇게 계산된 중력 가속도는 특정한 위치에서의 중력의 세기를 나타내며, 지구 표면 근처에서는 대략 9.81 m/s²로 일정하게 유지됩니다.'

In [17]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(재구성 증강 생성)는 정보 검색과 생성 모델을 결합한 접근 방식으로, 외부 데이터베이스에서 관련 정보를 검색한 후 이를 활용해 더욱 정확하고 풍부한 응답을 생성하는 방법입니다. 주로 자연어 처리에서 사용되며, 질문에 대한 답변을 생성할 때 필요한 정보를 검색하여 모델의 성능을 향상시킵니다.'